In [1]:
import os
import numpy as np
import pandas as pd
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import Dataset, DataLoader
from torchvision import transforms, models, datasets
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score
from PIL import Image
import torch.nn.functional as F

In [2]:
# Set device
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

print(torch.cuda.is_available())

True


In [3]:
# Set random seed for reproducibility
np.random.seed(42)
torch.manual_seed(42)

# Set the path to your project folder
project_folder = '.'

# Set the path to the train and test data folders
train_data_dir = os.path.join(project_folder, "data", "train")
test_data_dir = os.path.join(project_folder, "data", "test")

# Load file names and labels for training
cat_images = [os.path.join(train_data_dir, "cat", file) for file in os.listdir(os.path.join(train_data_dir, "cat"))]
dog_images = [os.path.join(train_data_dir, "dog", file) for file in os.listdir(os.path.join(train_data_dir, "dog"))]

# Assign labels (0 for cat, 1 for dog)
labels = [0] * len(cat_images) + [1] * len(dog_images)

# Combine file names and labels
file_names = cat_images + dog_images

# Split the data into training and validation sets
train_files, val_files, train_labels, val_labels = train_test_split(file_names, labels, test_size=0.2, random_state=42)

# Image data augmentation for training
train_transform = transforms.Compose([
    transforms.RandomResizedCrop(224),
    transforms.RandomHorizontalFlip(),
    transforms.ToTensor(),
])

# Image data augmentation for validation
val_transform = transforms.Compose([
    transforms.Resize(256),
    transforms.CenterCrop(224),
    transforms.ToTensor(),
])

In [18]:
class CustomDataset(Dataset):
    def __init__(self, file_paths, labels, transform=None):
        self.file_paths = file_paths
        self.labels = labels
        self.transform = transform

    def __len__(self):
        return len(self.file_paths)

    def __getitem__(self, idx):
        img_path = self.file_paths[idx]
        img = Image.open(img_path).convert('RGB')

        if self.transform:
            img = self.transform(img)

        # Check if labels are provided
        if self.labels is not None:
            label = torch.tensor(self.labels[idx], dtype=torch.float32)
            return img, label
        else:
            return img

In [5]:
# Define batch size
batch_size = 32

# Create custom datasets and dataloaders
train_dataset = CustomDataset(train_files, train_labels, transform=train_transform)
val_dataset = CustomDataset(val_files, val_labels, transform=val_transform)

train_loader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True)
val_loader = DataLoader(val_dataset, batch_size=batch_size, shuffle=False)

In [6]:
# Define and train the Convolutional Neural Network (CNN) model
class CNNModel(nn.Module):
    def __init__(self):
        super(CNNModel, self).__init__()
        self.conv1 = nn.Conv2d(3, 32, kernel_size=3, stride=1, padding=1)
        self.pool = nn.MaxPool2d(kernel_size=2, stride=2, padding=0)
        self.conv2 = nn.Conv2d(32, 64, kernel_size=3, stride=1, padding=1)
        self.conv3 = nn.Conv2d(64, 128, kernel_size=3, stride=1, padding=1)
        self.fc1 = nn.Linear(128 * 28 * 28, 512)
        self.dropout = nn.Dropout(0.5)
        self.fc2 = nn.Linear(512, 1)

    def forward(self, x):
        x = self.pool(F.relu(self.conv1(x)))
        x = self.pool(F.relu(self.conv2(x)))
        x = self.pool(F.relu(self.conv3(x)))
        x = x.view(-1, 128 * 28 * 28)
        x = F.relu(self.fc1(x))
        x = self.dropout(x)
        x = torch.sigmoid(self.fc2(x))
        return x

model_cnn = CNNModel()

# Display the model summary
print(model_cnn)

model_cnn.to(device)

CNNModel(
  (conv1): Conv2d(3, 32, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
  (pool): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
  (conv2): Conv2d(32, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
  (conv3): Conv2d(64, 128, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
  (fc1): Linear(in_features=100352, out_features=512, bias=True)
  (dropout): Dropout(p=0.5, inplace=False)
  (fc2): Linear(in_features=512, out_features=1, bias=True)
)


CNNModel(
  (conv1): Conv2d(3, 32, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
  (pool): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
  (conv2): Conv2d(32, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
  (conv3): Conv2d(64, 128, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
  (fc1): Linear(in_features=100352, out_features=512, bias=True)
  (dropout): Dropout(p=0.5, inplace=False)
  (fc2): Linear(in_features=512, out_features=1, bias=True)
)

In [7]:
# Loss and optimizer
criterion = nn.BCELoss()
optimizer_cnn = optim.Adam(model_cnn.parameters(), lr=0.001)

In [8]:

# Train the CNN model
num_epochs = 10
for epoch in range(num_epochs):
    model_cnn.train()
    for inputs, labels in train_loader:
        inputs, labels = inputs.to(device), labels.to(device)
        optimizer_cnn.zero_grad()
        outputs = model_cnn(inputs)
        loss = criterion(outputs, labels.view(-1, 1))
        loss.backward()
        optimizer_cnn.step()

    # Validation
    model_cnn.eval()
    with torch.no_grad():
        val_predictions_cnn = []
        val_labels_list = []
        for inputs, labels in val_loader:
            inputs, labels = inputs.to(device), labels.to(device)
            val_outputs = model_cnn(inputs)
            val_predictions_cnn.extend(val_outputs.cpu().numpy())
            val_labels_list.extend(labels.cpu().numpy())

        val_predictions_cnn = np.round(np.array(val_predictions_cnn)).flatten()
        accuracy_cnn = accuracy_score(val_labels_list, val_predictions_cnn)
        print(f"Epoch [{epoch+1}/{num_epochs}], Validation Accuracy (CNN): {accuracy_cnn}")

Epoch [1/10], Validation Accuracy (CNN): 0.6192
Epoch [2/10], Validation Accuracy (CNN): 0.6164
Epoch [3/10], Validation Accuracy (CNN): 0.7128
Epoch [4/10], Validation Accuracy (CNN): 0.7478
Epoch [5/10], Validation Accuracy (CNN): 0.7668
Epoch [6/10], Validation Accuracy (CNN): 0.7902
Epoch [7/10], Validation Accuracy (CNN): 0.8
Epoch [8/10], Validation Accuracy (CNN): 0.8132
Epoch [9/10], Validation Accuracy (CNN): 0.8148
Epoch [10/10], Validation Accuracy (CNN): 0.8244


In [9]:
# Define and train the Transfer Learning model using MobileNet
class TransferLearningModel(nn.Module):
    def __init__(self):
        super(TransferLearningModel, self).__init__()
        mobilenet = models.mobilenet_v2(pretrained=True)
        self.features = mobilenet.features
        self.avgpool = nn.AdaptiveAvgPool2d(1)
        self.fc = nn.Linear(1280, 1)

    def forward(self, x):
        x = self.features(x)
        x = self.avgpool(x)
        x = x.view(x.size(0), -1)
        x = torch.sigmoid(self.fc(x))
        return x

model_transfer = TransferLearningModel()

# Display the model summary
print(model_transfer)

# Set device
model_transfer.to(device)

# Loss and optimizer
optimizer_transfer = optim.Adam(model_transfer.parameters(), lr=0.001)

/home/nurbo/Desktop/Fall 2023/ROBT 407 Machine Learning with Applications/final_project/venv/lib/python3.10/site-packages/torchvision/models/_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
  warnings.warn(
/home/nurbo/Desktop/Fall 2023/ROBT 407 Machine Learning with Applications/final_project/venv/lib/python3.10/site-packages/torchvision/models/_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=MobileNet_V2_Weights.IMAGENET1K_V1`. You can also use `weights=MobileNet_V2_Weights.DEFAULT` to get the most up-to-date weights.
  warnings.warn(msg)
Downloading: "https://download.pytorch.org/models/mobilenet_v2-b0353104.pth" to /home/nurbo/.cache/torch/hub/checkpoints/mobilenet_v2-b0353104.pth
100.0%


TransferLearningModel(
  (features): Sequential(
    (0): Conv2dNormActivation(
      (0): Conv2d(3, 32, kernel_size=(3, 3), stride=(2, 2), padding=(1, 1), bias=False)
      (1): BatchNorm2d(32, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (2): ReLU6(inplace=True)
    )
    (1): InvertedResidual(
      (conv): Sequential(
        (0): Conv2dNormActivation(
          (0): Conv2d(32, 32, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), groups=32, bias=False)
          (1): BatchNorm2d(32, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
          (2): ReLU6(inplace=True)
        )
        (1): Conv2d(32, 16, kernel_size=(1, 1), stride=(1, 1), bias=False)
        (2): BatchNorm2d(16, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      )
    )
    (2): InvertedResidual(
      (conv): Sequential(
        (0): Conv2dNormActivation(
          (0): Conv2d(16, 96, kernel_size=(1, 1), stride=(1, 1), bias=False)
          (1): BatchNorm2

In [10]:
# Train the Transfer Learning model
for epoch in range(num_epochs):
    model_transfer.train()
    for inputs, labels in train_loader:
        inputs, labels = inputs.to(device), labels.to(device)
        optimizer_transfer.zero_grad()
        outputs = model_transfer(inputs)
        loss = criterion(outputs, labels.view(-1, 1))
        loss.backward()
        optimizer_transfer.step()

    # Validation
    model_transfer.eval()
    with torch.no_grad():
        val_predictions_transfer = []
        val_labels_list = []
        for inputs, labels in val_loader:
            inputs, labels = inputs.to(device), labels.to(device)
            val_outputs = model_transfer(inputs)
            val_predictions_transfer.extend(val_outputs.cpu().numpy())
            val_labels_list.extend(labels.cpu().numpy())

        val_predictions_transfer = np.round(np.array(val_predictions_transfer)).flatten()
        accuracy_transfer = accuracy_score(val_labels_list, val_predictions_transfer)
        print(f"Epoch [{epoch+1}/{num_epochs}], Validation Accuracy (Transfer Learning): {accuracy_transfer}")

Epoch [1/10], Validation Accuracy (Transfer Learning): 0.9552
Epoch [2/10], Validation Accuracy (Transfer Learning): 0.9606
Epoch [3/10], Validation Accuracy (Transfer Learning): 0.9634
Epoch [4/10], Validation Accuracy (Transfer Learning): 0.9512
Epoch [5/10], Validation Accuracy (Transfer Learning): 0.9698
Epoch [6/10], Validation Accuracy (Transfer Learning): 0.9616
Epoch [7/10], Validation Accuracy (Transfer Learning): 0.9646
Epoch [8/10], Validation Accuracy (Transfer Learning): 0.9702
Epoch [9/10], Validation Accuracy (Transfer Learning): 0.9686
Epoch [10/10], Validation Accuracy (Transfer Learning): 0.9674


In [21]:
# Choose the best model (the one with higher validation accuracy)
chosen_model = model_cnn if accuracy_cnn > accuracy_transfer else model_transfer

In [28]:
# Generate test data
test_files = [os.path.join(test_data_dir, file) for file in os.listdir(test_data_dir)]

test_transform = transforms.Compose([
    transforms.Resize(256),
    transforms.CenterCrop(224),
    transforms.ToTensor(),
])

test_dataset = CustomDataset(test_files, labels=None, transform=test_transform)
test_loader = DataLoader(test_dataset, batch_size=batch_size, shuffle=False)

In [29]:
# Make predictions on the test set
test_predictions = []
chosen_model.eval()
with torch.no_grad():
    for inputs in test_loader:
        inputs = inputs.to(device)
        outputs = chosen_model(inputs)
        print(outputs)
        test_predictions.extend(outputs.cpu().numpy())

tensor([[9.9871e-01],
        [7.1621e-03],
        [8.2805e-04],
        [9.9948e-01],
        [2.0762e-01],
        [8.2640e-06],
        [1.2828e-03],
        [4.0054e-01],
        [8.0685e-06],
        [3.7551e-05],
        [9.1862e-05],
        [9.9992e-01],
        [1.5457e-01],
        [5.9932e-07],
        [2.5684e-05],
        [9.9569e-01],
        [5.1815e-03],
        [1.0000e+00],
        [2.0033e-01],
        [7.3130e-08],
        [9.9365e-01],
        [9.9874e-01],
        [9.9962e-01],
        [2.4866e-02],
        [2.1676e-07],
        [4.3643e-06],
        [1.0000e+00],
        [4.3277e-05],
        [2.8289e-07],
        [4.1692e-05],
        [3.7681e-04],
        [1.4621e-04]], device='cuda:0')
tensor([[2.7289e-07],
        [9.9999e-01],
        [3.4894e-05],
        [2.9102e-05],
        [2.6788e-06],
        [1.0000e+00],
        [6.9883e-05],
        [5.2247e-05],
        [9.9819e-01],
        [1.5126e-02],
        [4.6611e-01],
        [9.9995e-01],
        [9.975

In [30]:
# Create a DataFrame with image IDs and corresponding probabilities
image_ids = [int(file.split('.')[0]) for file in os.listdir(test_data_dir)]
submission_df = pd.DataFrame({'id': image_ids, 'label': np.array(test_predictions).flatten()})

# Save the DataFrame to a CSV file
submission_df.to_csv('submission.csv', index=False)